In [0]:
!pip install soundfile
!pip install librosa

In [0]:
from google.colab import drive
#import soundfile
import librosa
import pickle
from sklearn.svm import SVC 
from sklearn.naive_bayes import GaussianNB
import numpy as np
import pandas as pd
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [2]:

drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [3]:
#создаю датафрейм для аудио
data = pd.DataFrame (columns=['Name_audio','Audio', 'Feature_Tone', 'Tone.Ravel'])
data

,Name_audio,Audio,Feature_Tone,Tone.Ravel


In [4]:
#получаю список названий файлов в папке с аудио
path = "/content/gdrive/My Drive/audio_train/train/"
name_files = os.listdir(path)
data['Name_audio'] = name_files #записываю названия в датафрейм
data

,Name_audio,Audio,Feature_Tone,Tone.Ravel
0,f9b737c1d2083b6a5a85.wav,NaN,NaN,NaN
1,f9b80a8a5903c7006f7a.wav,NaN,NaN,NaN
2,70b9edbb2fb73de11b45.wav,NaN,NaN,NaN
3,f98ff11b6ab64c31ac6d.wav,NaN,NaN,NaN
4,f98de3d40918553d4599.wav,NaN,NaN,NaN
...,...,...,...,...
5678,255da5ed7dafcb5f2e64.wav,NaN,NaN,NaN
5679,0fa7cca006b8c6a9b969.wav,NaN,NaN,NaN
5680,252b858b428897138f84.wav,NaN,NaN,NaN
5681,250b6832901c953a672d.wav,NaN,NaN,NaN


In [0]:
data['Audio'][5011]

nan

In [0]:
#прохожусь по всем аудио и в датафрейм записываю полученные значения ?амплитуд? с помощью librosa
for i in range(3832,len(name_files)):
  data['Audio'][i], fs = librosa.load(path + name_files[i] )
  #if i%500 == 0:
  print(i)
data.head()

In [0]:
#сохранить в формате csv

In [0]:
data.to_csv('/content/gdrive/My Drive/data_Audio.csv')

In [0]:
data = pd.read_csv('/content/gdrive/My Drive/data_Audio.csv')
data.head()

,Unnamed: 0,Name_audio,Audio,Feature_Tone,Tone.Ravel
0,0,f9b737c1d2083b6a5a85.wav,[-0.10846483 0.05404515 0.26760167 ... 0.00...,NaN,NaN
1,1,f9b80a8a5903c7006f7a.wav,[-8.1808947e-04 -1.7469880e-03 -2.1493509e-03 ...,NaN,NaN
2,2,70b9edbb2fb73de11b45.wav,[0.00044923 0.00055444 0.0004087 ... 0.000447...,NaN,NaN
3,3,f98ff11b6ab64c31ac6d.wav,[ 2.64214294e-04 1.18508644e-04 -1.46041668e-...,NaN,NaN
4,4,f98de3d40918553d4599.wav,[-1.8483361e-03 -3.1981971e-03 -3.1196314e-03 ...,NaN,NaN


In [0]:
#сохраняю значение амплитуд в формате pickle
with open('/content/gdrive/My Drive/Colab Notebooks/audio.pickle', 'wb') as f:
...     pickle.dump(list(data['Audio']), f)

In [0]:
#выгружаю из pickle значения амплитуд
with open('/content/gdrive/My Drive/Colab Notebooks/audio.pickle', 'rb') as f:
...     Audio = pickle.load(f)

In [10]:
data['Audio'] = Audio
data.head()

,Name_audio,Audio,Feature_Tone,Tone.Ravel
0,f9b737c1d2083b6a5a85.wav,"[-0.10846483, 0.05404515, 0.26760167, 0.026163...",NaN,NaN
1,f9b80a8a5903c7006f7a.wav,"[-0.00081808947, -0.001746988, -0.002149351, -...",NaN,NaN
2,70b9edbb2fb73de11b45.wav,"[0.000449231, 0.0005544413, 0.00040870407, 9.5...",NaN,NaN
3,f98ff11b6ab64c31ac6d.wav,"[0.0002642143, 0.000118508644, -0.00014604167,...",NaN,NaN
4,f98de3d40918553d4599.wav,"[-0.0018483361, -0.003198197, -0.0031196314, -...",NaN,NaN


In [0]:
#получение признаков tone
for i in range(len(name_files)):
  print(i)
  data['Feature_Tone'][i] = librosa.feature.chroma_stft(data['Audio'][i])
data.head()

In [0]:
#сохраняю признаки в формате pickle
with open('/content/gdrive/My Drive/Colab Notebooks/Feature_Tone.pickle', 'wb') as f:
...     pickle.dump(list(data['Feature_Tone']), f)

In [0]:
#выгружаю из pickle признаки tone
with open('/content/gdrive/My Drive/Colab Notebooks/Feature_Tone.pickle', 'rb') as f:
...     Feature_Tone = pickle.load(f)

In [14]:
data['Feature_Tone'] = Feature_Tone
data.head()

,Name_audio,Audio,Feature_Tone,Tone.Ravel
0,f9b737c1d2083b6a5a85.wav,"[-0.10846483, 0.05404515, 0.26760167, 0.026163...","[[0.028820622776791868, 0.035540443717563326, ...",NaN
1,f9b80a8a5903c7006f7a.wav,"[-0.00081808947, -0.001746988, -0.002149351, -...","[[0.43014247714434317, 0.21300254030154597, 0....",NaN
2,70b9edbb2fb73de11b45.wav,"[0.000449231, 0.0005544413, 0.00040870407, 9.5...","[[0.26519665180603347, 0.2001198163939377, 0.1...",NaN
3,f98ff11b6ab64c31ac6d.wav,"[0.0002642143, 0.000118508644, -0.00014604167,...","[[0.003857842635125953, 0.0006431823830839563,...",NaN
4,f98de3d40918553d4599.wav,"[-0.0018483361, -0.003198197, -0.0031196314, -...","[[0.12975261057595686, 0.3552813475665379, 0.0...",NaN


In [0]:
#переформировываю масив в одномерный для построения классификаторв
for i in range(len(name_files)):
  data['Tone.Ravel'][i] = data['Feature_Tone'][i].ravel()
data.head()

In [0]:
with open('/content/gdrive/My Drive/Colab Notebooks/Tone_Ravel.pickle', 'wb') as f:
...     pickle.dump(list(data['Tone.Ravel']), f)

In [0]:
with open('/content/gdrive/My Drive/Colab Notebooks/Tone_Ravel.pickle', 'rb') as f:
...     Tone_Ravel = pickle.load(f)

In [16]:
data['Tone.Ravel'] = Tone_Ravel
data.head()


,Name_audio,Audio,Feature_Tone,Tone.Ravel
0,f9b737c1d2083b6a5a85.wav,"[-0.10846483, 0.05404515, 0.26760167, 0.026163...","[[0.028820622776791868, 0.035540443717563326, ...","[0.028820622776791868, 0.035540443717563326, 0..."
1,f9b80a8a5903c7006f7a.wav,"[-0.00081808947, -0.001746988, -0.002149351, -...","[[0.43014247714434317, 0.21300254030154597, 0....","[0.43014247714434317, 0.21300254030154597, 0.1..."
2,70b9edbb2fb73de11b45.wav,"[0.000449231, 0.0005544413, 0.00040870407, 9.5...","[[0.26519665180603347, 0.2001198163939377, 0.1...","[0.26519665180603347, 0.2001198163939377, 0.17..."
3,f98ff11b6ab64c31ac6d.wav,"[0.0002642143, 0.000118508644, -0.00014604167,...","[[0.003857842635125953, 0.0006431823830839563,...","[0.003857842635125953, 0.0006431823830839563, ..."
4,f98de3d40918553d4599.wav,"[-0.0018483361, -0.003198197, -0.0031196314, -...","[[0.12975261057595686, 0.3552813475665379, 0.0...","[0.12975261057595686, 0.3552813475665379, 0.01..."


In [0]:
#привожу признаки к одной длине
#определяю максимальную длину
max_len = np.max(list(map(len,list(data['Tone.Ravel']))))
for i in range(len(data)):
  #создаю нулевой массив с недостающими нулями
  zero = np.zeros(max_len - len(data['Tone.Ravel'][i]), dtype=int)
  #добавляю нулевой массив в конец
  data['Tone.Ravel'][i] = np.hstack((data['Tone.Ravel'][i], zero))



In [19]:
#загружаю лейблы
labels = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/train.csv')
labels.head()

,fname,label
0,8bcbcc394ba64fe85ed4.wav,Finger_snapping
1,00d77b917e241afa06f1.wav,Squeak
2,17bb93b73b8e79234cb3.wav,Electric_piano
3,7d5c7a40a936136da55e.wav,Harmonica
4,17e0ee7565a33d6c2326.wav,Snare_drum


In [0]:
#переименовываю название колонки для того чтобы потом объеденить признаки и лейблы в один дата фрейм
labels.rename(columns={'fname': 'Name_audio'}, inplace=True)

In [173]:
#объединяю 2 дата фрейма
all_data = data.merge(labels)
all_data.head()

,Name_audio,Audio,Feature_Tone,Tone.Ravel,label
0,f9b737c1d2083b6a5a85.wav,"[-0.10846483, 0.05404515, 0.26760167, 0.026163...","[[0.028820622776791868, 0.035540443717563326, ...","[0.028820622776791868, 0.035540443717563326, 0...",Hi-hat
1,f9b80a8a5903c7006f7a.wav,"[-0.00081808947, -0.001746988, -0.002149351, -...","[[0.43014247714434317, 0.21300254030154597, 0....","[0.43014247714434317, 0.21300254030154597, 0.1...",Double_bass
2,70b9edbb2fb73de11b45.wav,"[0.000449231, 0.0005544413, 0.00040870407, 9.5...","[[0.26519665180603347, 0.2001198163939377, 0.1...","[0.26519665180603347, 0.2001198163939377, 0.17...",Applause
3,f98ff11b6ab64c31ac6d.wav,"[0.0002642143, 0.000118508644, -0.00014604167,...","[[0.003857842635125953, 0.0006431823830839563,...","[0.003857842635125953, 0.0006431823830839563, ...",Oboe
4,f98de3d40918553d4599.wav,"[-0.0018483361, -0.003198197, -0.0031196314, -...","[[0.12975261057595686, 0.3552813475665379, 0.0...","[0.12975261057595686, 0.3552813475665379, 0.01...",Flute


In [174]:
#получаю уникальные значения лейблов
unique_labels = np.unique(all_data['label'])
print(unique_labels )

['Acoustic_guitar' 'Applause' 'Bark' 'Bass_drum' 'Burping_or_eructation'
 'Bus' 'Cello' 'Chime' 'Clarinet' 'Computer_keyboard' 'Cough' 'Cowbell'
 'Double_bass' 'Drawer_open_or_close' 'Electric_piano' 'Fart'
 'Finger_snapping' 'Fireworks' 'Flute' 'Glockenspiel' 'Gong'
 'Gunshot_or_gunfire' 'Harmonica' 'Hi-hat' 'Keys_jangling' 'Knock'
 'Laughter' 'Meow' 'Microwave_oven' 'Oboe' 'Saxophone' 'Scissors'
 'Shatter' 'Snare_drum' 'Squeak' 'Tambourine' 'Tearing' 'Telephone'
 'Trumpet' 'Violin_or_fiddle' 'Writing']


In [0]:
#разделяю на трейн и тест все данные в проморции 70 на 30 для каждого лейбла
X_train,  X_test,  Y_train,  Y_test = [], [], [], []
for i in range(len(unique_labels)):
  # получаю трейн и тест выборки для каждого лейбла
  X_train_one_label,  X_test_one_label,  Y_train_one_label,  Y_test_one_label =  train_test_split(list(all_data[all_data['label'] == unique_labels[i]]['Tone.Ravel']),  list(all_data[all_data['label'] == unique_labels[i]]['label']),  test_size = 0.3)
  
  # добавляю их в общий массив
  X_train.extend(X_train_one_label)
  X_test.extend(X_test_one_label)
  Y_train.extend(Y_train_one_label)
  Y_test.extend(Y_test_one_label)
  


In [176]:
len(Y_train)

3960

In [177]:
len(Y_test)

1723

In [0]:
#наивный Байес
nb = GaussianNB()
nb.fit( X_train, Y_train)
Y_predict = nb.predict(X_test)

In [182]:
print(classification_report(Y_test,Y_predict, digits = 4))

                       precision    recall  f1-score   support

      Acoustic_guitar     0.0000    0.0000    0.0000        57
             Applause     0.0000    0.0000    0.0000        55
                 Bark     0.0000    0.0000    0.0000        45
            Bass_drum     0.0000    0.0000    0.0000        55
Burping_or_eructation     0.0000    0.0000    0.0000        34
                  Bus     0.0606    0.1000    0.0755        20
                Cello     0.0000    0.0000    0.0000        56
                Chime     0.0000    0.0000    0.0000        24
             Clarinet     0.0000    0.0000    0.0000        53
    Computer_keyboard     0.0185    0.0500    0.0270        20
                Cough     0.0323    0.0408    0.0360        49
              Cowbell     0.0000    0.0000    0.0000        36
          Double_bass     0.0432    0.9655    0.0827        58
 Drawer_open_or_close     0.0000    0.0000    0.0000        29
       Electric_piano     0.0000    0.0000    0.0000  

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
